<div style="font-size: 200%; font-weight: bold; color: maroon;">Grafo de una empresa en Twitter</div>
<!-- Paulo Villegas, 2021 - v. 1.2 -->

Cargamos un grafo extraído de Twitter. Contiene las cuentas que interactúan (contestaciones, _retweets_ ) con las cuentas corporativas de Telefónica

In [ ]:
spark

# Leer datos

Los datos de nodos y enlaces están almacenados en ficheros CSV. Creamos dos DataFrames con ellos.

In [ ]:
nodes = spark.read.csv('DATA/twitter-tef/nodes.csv.bz2', inferSchema=True, header=True, sep=';')

In [ ]:
nodes.printSchema()

In [ ]:
edges = spark.read.csv('DATA/twitter-tef/edges.csv.bz2', inferSchema=True, header=True, sep=';')

In [ ]:
edges.printSchema()

In [ ]:
edges.limit(10).toPandas()

Tenemos un problema: el campo `Weight` es un número, pero usa la coma como separador decimal, por lo que Spark lo ha tomado como si fuera una cadena. Corregimos eso cambiando la coma por un punto, y haciendo un _cast_ al tipo que queremos.

In [ ]:
import pyspark.sql.functions as F

edges = edges.withColumn('Weight', F.regexp_replace('Weight', ',', '.').cast('float'))

# Creación del grafo

Ahora creamos el objeto GraphFrames

In [ ]:
import graphframes as gf

In [ ]:
g = gf.GraphFrame(nodes.withColumnRenamed('Id','id'), 
                  edges.withColumnRenamed('Source', 'src').withColumnRenamed('Target', 'dst'))

In [ ]:
g.edges.count()

In [ ]:
g.vertices.count()

In [ ]:
g.vertices.limit(10).toPandas()

In [ ]:
g.edges.limit(10).toPandas()

In [ ]:
g.edges.select('Relation').distinct().toPandas()

# PageRank y grados

Cruzamos el valor de PageRank de cada nodo con su grado, para poder comparar ambos rankings.

Nótese que los valores estarán sesgados, porque no estamos usando el grafo compleot de Twitter, sino un subgrafo centrado en Telefónica y sus contactos.

In [ ]:
%%time
pg = g.pageRank(maxIter=5)

In [ ]:
pg_ord = pg.vertices.orderBy('pagerank', ascending=False)

In [ ]:
pg_ord.toPandas()

In [ ]:
pgd = pg_ord.join(g.degrees, 'id')

In [ ]:
pgd.toPandas()

# Búsqueda de patrones

Buscamos candidatos a seguidores: cuentas que son amigas de una cuenta que ya es seguidora de _movistar-es_

In [ ]:
motifs = g.find("(a)-[e1]->(b); (b)-[e2]->(c); !(a)-[]->(c) ")

In [ ]:
motifs.count()

In [ ]:
motifs2 = motifs.filter( 'a.id == "movistar_es"')

In [ ]:
motifs2.count()

In [ ]:
motifs3 = motifs2.filter( 'e1.Relation == "FAN" AND e2.Relation == "FRIEND"' )

In [ ]:
motifs3.toPandas()

Ahora buscaremos cuentas influyentes (alto número de seguidores) que estén enlazadas a la cuenta _movistar-es_

In [ ]:
influencers = g.find("(a)-[]->(b)").filter('a.id == "movistar_es" AND b.Num_Followers > 1000')

In [ ]:
influencers.toPandas()

# Representación gráfica
Como el grafo no es muy grande, podemos tratar de representarlo gráficamente. Para ello lo traemos desde Spark a local, y lo cargamos en NetworkX

In [ ]:
pd_edges = g.edges.toPandas()
pd_nodes = g.vertices.toPandas()

import networkx as nx
gnx = nx.from_pandas_edgelist(pd_edges,'src','dst', ['Weight', 'Relation'])
for n in pd_nodes.itertuples(index=False):
    gnx.add_node(n.id, **n._asdict())

In [ ]:
pd_edges[pd_edges.dst == 'telefonica']

Y ahora lo dibujamos

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

color = ['red' if n.id == 'telefonica' else 'black' for n in pd_nodes.itertuples()]
size = [50 if n.id == 'telefonica' else 5 for n in pd_nodes.itertuples()]

fig, ax = plt.subplots(1, 1, figsize=(18, 12))
pos = nx.spring_layout(gnx, weight='Weight', seed=3410)
nx.draw_networkx(gnx, pos=pos, ax=ax, with_labels=False, node_color=color,
                 node_size=size, width=.5, alpha=0.75, edge_color='#add8e6')
ax.set_axis_off()